Il faut mettre en place un script python pour récupérer automatiquement des données depuis le web. Dans mon cas j'ai un fichier urls.txt qui contient 19 urls que je vais utiliser.

Pour l'instant je vais utiliser ce notebook pour extraire le contenu textuel pour une page, et je vais ensuite adapter cela au reste des pages.

In [2]:
import requests

r = requests.get('https://www.mediafax.ro')
print(r.status_code)

200


In [3]:
print(r.content)

b'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ro" xml:lang="ro">\n\n    <head>\n\n        <meta name="google-site-verification" content="F84kaIgAlvwDvz__IYwu4zUHAExspfmJjzJ6ArraBh0" />\n        <meta name="google-site-verification" content="83qB9dMT3PdyVPeSXH8QpECV9DvGO-haV-FVBsQdLIo" />\n        <meta name="google-site-verification" content="llssVK3V5VYLD62XCUltG3dMTX62AlqwOWEJUGgNJmk" />\n\n        <meta http-equiv="Content-type" content="text/html;charset=UTF-8" />\n        <link rel="shortcut icon" href="/favicon.ico" />\n        <link rel="stylesheet" type="text/css" href="/style.css?v=10,11" media="screen" />\n\n         <script type="text/javascript">  </script>         <title>\xc5\x9etiri de ultima ora si ultimele \xc5\x9ftiri - \xc5\x9ftiri online - Mediafax\n</title>\n\n\t\t<meta property="fb:app_id" content="816679868905651"/>\n\t<meta property="fb:pages" content="109837159037642" />\n\t<meta property="og:site_name" content="Mediafax.ro" />\n\t<meta pro

In [14]:
import lxml.html

root = lxml.html.fromstring(r.content)
a_tags = root.xpath('//*[@id="content"]/div[3]/ul/li/p/a')
print(a_tags)

[<Element a at 0x1121c0780>, <Element a at 0x1121c0a00>, <Element a at 0x1121c0140>, <Element a at 0x1121c09b0>, <Element a at 0x1121c1130>, <Element a at 0x1121c1180>, <Element a at 0x1121c11d0>, <Element a at 0x1121c1310>, <Element a at 0x1121c1360>, <Element a at 0x1121c1400>, <Element a at 0x1121c13b0>, <Element a at 0x1121c1450>, <Element a at 0x1121c14a0>, <Element a at 0x1121c14f0>, <Element a at 0x1121c0cd0>, <Element a at 0x1121c0910>]


In [20]:
url_list = []
for element in a_tags:
    href = element.attrib['href']
    if href.startswith('http://') or href.startswith('https://'): # if it's from a different website
        # url_list.append(href)
        pass
    elif href.startswith('/'):
        url_list.append('https://www.mediafax.ro' + href)
    else:
        url_list.append('https://www.mediafax.ro/' + href)
    
print(url_list)


['https://www.mediafax.ro/externe/secretarul-american-al-trezoreriei-sua-va-sanctiona-china-daca-ajuta-rusia-in-razboiul-din-ucraina-22329753', 'https://www.mediafax.ro/politic/catalin-cirstoiu-am-fost-transformat-in-doua-saptamani-intr-un-inamic-public-numarul-unu-omul-negru-22329745', 'https://www.mediafax.ro/politic/rares-bogdan-spune-ca-bucurestiul-este-mai-sigur-noaptea-pe-strada-decat-londra-paris-roma-berlin-madrid-datorita-ministrilor-pnl-de-la-interne-22329472', 'https://www.mediafax.ro/economic/la-vara-vom-putea-circula-pe-autostrada-de-la-pitesti-la-constanta-pentru-prima-data-bucurestiul-nu-va-mai-fi-o-piedica-in-transporturi-22329360', 'https://www.mediafax.ro/economic/comisia-europeana-aproba-reintroducerea-schemei-de-sprijinire-a-companiilor-romanesti-22329172', 'https://www.mediafax.ro/social/mina-marina-din-zona-tulcea-a-fost-ridicata-de-echipa-de-scafandri-pirotehnicieni-ai-mapn-iata-de-unde-provine-22329170', 'https://www.mediafax.ro/tehnologie/un-alt-razboi-dronele-

In [29]:
from collections import namedtuple

Item = namedtuple('Item', ['id', 'category', 'text'])
list_items = []

for url in url_list:
    article_id = url.split('-')[-1]
    category = url.split('/')[3]
    r = requests.get(url)
    url_root = lxml.html.fromstring(r.content)
    paragraphs = url_root.xpath('//*[@id="article_text_content"]/div[1]/p')
    text = ''
    for paragraph in paragraphs:
        text += paragraph.text_content().strip() + ' '
    item = Item(article_id, category, text)
    list_items.append(item)


Item(id='22329753', category='externe', text='Secretarul Trezoreriei de stat a SUA, Janet Yellen, a declarat, în cadrul unui interviu pentru CNBC, că Washingtonul va putea lua măsuri restrictive împotriva Chinei dacă aceasta va susţine efortul   de război al Rusiei în invazia din Ucraina. ,,Tot ceea ce implică sprijinirea armatei ruse în războiul lor brutal împotriva Ucrainei este inacceptabil pentru noi şi avem capacitatea de a sancţiona   acest lucru", a afirmat Yellen. Potrivit CNBC, Yellen a declarat că   Departamentul Trezoreriei nu a impus asemenea sancţiuni până în prezent, întrucât relaţia comercială dintre China şi Rusia este considerată de SUA ca fiind neproblematică în mare parte. Totuşi,   secretarul Trezoreriei americane a subliniat faptul că furnizarea de ajutor militar pentru Moscova, din partea Beijingului, ar putea să conducă la o serie de sancţiuni din partea Trezoreriei.  Declaraţiile au avut loc în contextul în care Washingtonul încearcă să stabilizeze relaţia diplo

In [32]:
# create the raw csv file 
import csv

with open('../../data/raw/romanian_news.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(Item._fields)
    for item in list_items:
        writer.writerow(item)

In [44]:
import spacy
nlp = spacy.load('ro_core_news_sm')

list_clean_items = []
for item in list_items:
    doc = nlp(item.text)
    ne = [ent.text for ent in doc.ents]
    clean_text = []
    for token in doc:
        if token.text in ne:
            clean_text.append("$NE$")
        else:
            clean_text.append(token.text)
    item = Item(item.id, item.category, ' '.join(clean_text))
    list_clean_items.append(item)

print(list_clean_items)

[Item(id='22329753', category='externe', text='Secretarul $NE$ de stat a $NE$ , Janet Yellen , a declarat , în cadrul unui interviu pentru $NE$ , că $NE$ va putea lua măsuri restrictive împotriva $NE$ dacă aceasta va susţine efortul    de război al $NE$ în invazia din $NE$ . , , Tot ceea ce implică sprijinirea armatei $NE$ în războiul lor brutal împotriva $NE$ este inacceptabil pentru noi şi avem capacitatea de a sancţiona    acest lucru " , a afirmat Yellen . Potrivit $NE$ , Yellen a declarat că    Departamentul $NE$ nu a impus asemenea sancţiuni până în prezent , întrucât relaţia comercială dintre $NE$ şi $NE$ este considerată de $NE$ ca fiind neproblematică în mare parte . Totuşi ,    $NE$ $NE$ $NE$ a subliniat faptul că furnizarea de ajutor militar pentru $NE$ , din partea $NE$ , ar putea să conducă la o serie de sancţiuni din partea $NE$ .   Declaraţiile au avut loc în contextul în care $NE$ încearcă să stabilizeze relaţia diplomatică tensionată dintre cele $NE$ superputeri rămase

In [45]:
# create the clean csv file 
import csv

with open('../../data/clean/romanian_news.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(Item._fields)
    for item in list_clean_items:
        writer.writerow(item)